# Урок 5. Сверточные нейронные сети для анализа текста.

Задание из 2-х частей.

Берем отзывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95
2. Берём предобученный word2vec и его эмбедингами инициализируем сетку, как влияет на качество?

#### 1. Учим conv сеть для классификации - выбить auc выше 0.95

In [175]:
import pandas as pd
import re
import numpy as np

from nltk import tokenize as tknz
from gensim.models import Word2Vec
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

##### Загрузка данных и подготовка

In [176]:
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [177]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [178]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [195]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

##### Формирование эмбеддинга

In [196]:
max_features = 2500
max_len = 200
num_classes = 1
batch_size = 64
epochs = 4
output_dim = 128

In [197]:
X_train

547                                              классно
1863       невозможно использовать на рутованный телефон
13949                      работать хорошо без нарекание
11501                                            отлично
6238     ян мочь понять почему заблокировать мой аккаунт
                              ...                       
5897                                        спсиб за всё
19812                                              супер
14734                                             удобно
4851                                               супер
19238                                         всё хорошо
Name: text, Length: 15798, dtype: object

In [198]:
text_vectorizer = TextVectorization(max_tokens=max_features,
                                    output_mode='int',
                                    output_sequence_length=max_len)
text_vectorizer.adapt(np.asarray(X_train))

X_train_v = text_vectorizer(X_train)
X_test_v = text_vectorizer(X_test)

##### Модель

In [199]:
model = Sequential()
model.add(Embedding(input_dim=max_features+1, output_dim=output_dim, input_length=max_len))
model.add(Conv1D(filters=output_dim, kernel_size=3, padding='same', activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [200]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='AUC')

##### Обучение

In [201]:
model.fit(X_train_v, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=False)

Epoch 1/4
247/247 [==============================] - 6s 21ms/step - loss: 0.3494 - auc: 0.7993
Epoch 2/4
247/247 [==============================] - 5s 21ms/step - loss: 0.1613 - auc: 0.9680
Epoch 3/4
247/247 [==============================] - 5s 21ms/step - loss: 0.1245 - auc: 0.9809
Epoch 4/4
247/247 [==============================] - 5s 21ms/step - loss: 0.0961 - auc: 0.9878


##### Результаты

In [202]:
roc_auc_score(y_test, model.predict(X_test_v, batch_size=batch_size, verbose=1))

62/62 [==============================] - 0s 5ms/step


0.9697320535151763

#### 2. Берём предобученный word2vec и его эмбедингами инициализируем сетку, как влияет на качество?

##### Формирование эмбеддинга

In [219]:
X_train_tok = []
for x in X_train:
    X_train_tok.append(tknz.word_tokenize(x, language="russian"))

In [220]:
X_train_tok[:3]

[['классно'],
 ['невозможно', 'использовать', 'на', 'рутованный', 'телефон'],
 ['работать', 'хорошо', 'без', 'нарекание']]

In [221]:
w2v_model = Word2Vec(min_count=3, size=output_dim, window=5)
w2v_model.build_vocab(X_train_tok)
w2v_model.train(X_train_tok, total_examples=len(X_train_tok), epochs=40)

(3120330, 4685360)

##### Модель

In [223]:
model2 = Sequential()
model2.add(w2v_model.wv.get_keras_embedding(train_embeddings=False))
model2.add(Conv1D(filters=output_dim, kernel_size=3, padding='same', activation='relu'))
model2.add(GlobalMaxPool1D())
model2.add(Dense(num_classes))
model2.add(Activation('sigmoid'))

In [224]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics='AUC')

##### Обучение

In [225]:
model2.fit(X_train_v, 
           y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           shuffle=False)

Epoch 1/4
247/247 [==============================] - 4s 16ms/step - loss: 0.2653 - auc: 0.9012
Epoch 2/4
247/247 [==============================] - 4s 16ms/step - loss: 0.1793 - auc: 0.9593
Epoch 3/4
247/247 [==============================] - 4s 16ms/step - loss: 0.1493 - auc: 0.9723
Epoch 4/4
247/247 [==============================] - 4s 16ms/step - loss: 0.1261 - auc: 0.9806


##### Результаты

In [226]:
roc_auc_score(y_test, model2.predict(X_test_v, batch_size=batch_size, verbose=1))

62/62 [==============================] - 0s 5ms/step


0.9525399943174097

##### С предобученным word2vec метрика немного упала с 0.9697320535151763 до 0.9525399943174097.